<a href="https://colab.research.google.com/github/brit228/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-2-validate-classification-problems/LS_DS_242_Validate_classification_problems_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 2.2MB/s 


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip bank-additional.zip

--2019-01-30 14:51:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/zip]
Saving to: ‘bank-additional.zip’

bank-additional.zip 100%[===================>] 434.15K   554KB/s    in 0.8s    

2019-01-30 14:51:54 (554 KB/s) - ‘bank-additional.zip’ saved [444572/444572]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/._bank-ad

In [35]:
import pandas as pd

df = pd.read_csv("bank-additional/bank-additional-full.csv", delimiter=";")

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [36]:
for i in df:
  print("{:50s}\t{}".format(i, df[i].dtype))

age                                               	int64
job                                               	object
marital                                           	object
education                                         	object
default                                           	object
housing                                           	object
loan                                              	object
contact                                           	object
month                                             	object
day_of_week                                       	object
duration                                          	int64
campaign                                          	int64
pdays                                             	int64
previous                                          	int64
poutcome                                          	object
emp.var.rate                                      	float64
cons.price.idx                                    	float64
cons.conf.idx    

In [0]:
y_col = "y"

train_ratio = 0.7
val_ratio = 0.15

shuffle = False
override_model = None
override_params = {}
override_scorer = None

n = 1
bin_cols = [c for c in df if df[c].dtype != 'object']

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

import numpy as np

model = None
params = {}
scorer = None
codes = []

if not override_model:
  if df[y_col].dtype == 'object':
    if len(df[y_col].astype('category').cat.categories) < 3 and df[y_col].isna().sum() == 0:
      m = {x: i for i,x in enumerate(df[y_col].astype('category').cat.categories)}
      n_col = "{}:{}".format(y_col, df[y_col].astype('category').cat.categories[-1])
      df[n_col] = df[y_col].map(m)
      df = df.drop(columns=[y_col], axis=1)
      codes = sorted(set(list(df[n_col].values)))
      y_col = [n_col]

      model = LogisticRegression
      args = [[np.power(10.0, i) for i in range(-10*n,10*n+1,1)]]
      params = {'solver': 'lbfgs'}
      scorer = accuracy_score
    else:
      df[y_col] = df[y_col].astype('category').cat.codes
      codes = sorted(set(list(df[y_col].values)))
      y_col = [y_col]
      
      model = LogisticRegression
      params = {'solver': 'lbfgs', 'multi_class': 'multinomial'}
      args = [[np.power(10.0, i) for i in range(-10*n,10*n+1,1)]]
      scorer = accuracy_score
  else:
    model = LogisticRegressionCV
    params = {}
    args = []
    scorer = r2_score
else:
  model = override_model
  
if override_params != {}:
  params = override_params
if override_scorer != None:
  scorer = override_scorer

In [0]:
from sklearn.model_selection import train_test_split

cols = [x for x in df if x not in y_col]

X_train = None
X_val = None
X_test = None
y_train = None
y_val = None
y_test = None

if train_ratio < 1.0:
  if val_ratio > 0.0:
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df[y_col], train_size=train_ratio+val_ratio, shuffle=shuffle)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=train_ratio/(train_ratio+val_ratio), shuffle=shuffle)
  else:
    X_train, X_test, y_train, y_test = train_test_split(df[cols], df[y_col], train_size=train_ratio, shuffle=shuffle)
    X_val = X_test.copy()
    y_val = y_test.copy()
else:
  X_val = X_train.copy()
  y_val = y_train.copy()
  X_test = X_train.copy()
  y_test = y_train.copy()

In [0]:
from sklearn.metrics import mean_squared_error
from scipy import stats

def calc_p(pipeline, model, X, y, y_pred):
  for i in range(y.shape[1]):
    params = np.append(model.intercept_[i],model.coef_[i])
    
    colX = pipeline.transform(X)
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(colX))

    var_b = mean_squared_error(y, y_pred)*(np.linalg.pinv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    return pd.Series([2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b], index=["Intercept"]+[x for x in colX])

In [0]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer, KBinsDiscretizer
from sklearn.externals.joblib import Parallel, delayed
from sklearn.pipeline import FeatureUnion, _fit_transform_one, _transform_one
from scipy import sparse


class DropFeatures(object):
  def __init__(self, cols=[]):
    self.cols = cols
    self.max = 0
  
  def set_params(self, **params):
    for key, value in params.items():
      if key == "cols":
        self.cols = value
    
  def transform(self, x):
    X = pd.DataFrame(x)
    self.max = len(list(X))
    return X[[i for i in X if i not in self.cols]]

  def fit(self, x, y=None):
    return self

class StandardScaler_df(object):
  def __init__(self):
    self.scale = StandardScaler()
    
  def transform(self, x):
    return pd.DataFrame(self.scale.transform(x), columns=list(x))
    
  def fit(self, x, y=None):
    return self.scale.fit(x)
  
class PowerTransformer_df(object):
  def __init__(self):
    self.power = PowerTransformer()
    
  def transform(self, x):
    return pd.DataFrame(self.power.transform(x), columns=list(x))
    
  def fit(self, x, y=None):
    return self.power.fit(x)
  
class KBinsDiscretizer_df(object):
  def __init__(self, bin_cols, n_bins=5):
    self.bin_cols = bin_cols
    self.n_bins = n_bins
    self.bins = KBinsDiscretizer(n_bins=n_bins, encode='onehot-dense')
    
  def transform(self, X):
    return pd.DataFrame(self.bins.transform(X), columns=["{}:{}".format(x,i) for i in range(self.n_bins) for x in self.bin_cols])
    
  def fit_transform(self, X, y):
    if type(X) != np.ndarray:
      self.bins.set_params(n_bins=[self.n_bins if x in self.bin_cols else 0 for x in X])
    else:
      self.bins.set_params(n_bins=[2]*X.shape[1])
    self.bins.fit(X, y)
    if type(X) == np.ndarray:
      return pd.DataFrame(self.bins.transform(X))
    return pd.DataFrame(self.bins.transform(X), columns=["{}:{}".format(x,i) for i in range(self.n_bins) for x in self.bin_cols])
  
  def fit(self, X, y=None):
    print(type(X))
    self.bins.set_params(n_bins=[self.n_bins if x in self.bin_cols else 0 for x in X])
    print(self.bins.n_bins)
    return self.bins.fit(X, y)
  
class PolynomialFeatures_df(object):
  def __init__(self, degree=2, interaction_only=False, include_bias=False):
    self.poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)
    
  def transform(self, x):
    return pd.DataFrame(self.poly.transform(x), columns=self.poly.get_feature_names(list(x)))
  
  def fit_transform(self, X, y):
    self.poly.fit(X, y)
    if type(X) == np.ndarray:
      return pd.DataFrame(self.poly.transform(X))
    return pd.DataFrame(self.poly.transform(X), columns=self.poly.get_feature_names(list(X)))
    
  def fit(self, x, y=None):
    return self.poly.fit(x, y)
  
class PandasFeatureUnion(FeatureUnion):
    def fit_transform(self, X, y=None, **fit_params):
        self._validate_transformers()
        result = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_transform_one)(
                transformer=trans,
                X=X,
                y=y,
                weight=weight,
                **fit_params)
            for name, trans, weight in self._iter())

        if not result:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        Xs, transformers = zip(*result)
        self._update_transformer_list(transformers)
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

    def merge_dataframes_by_column(self, Xs):
        return pd.concat(Xs, axis="columns", copy=False)

    def transform(self, X):
        Xs = Parallel(n_jobs=self.n_jobs)(
            delayed(_transform_one)(
                transformer=trans,
                X=X,
                y=None,
                weight=weight)
            for name, trans, weight in self._iter())
        if not Xs:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

In [112]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
import category_encoders as ce

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

def predict(pipe,mod,x):
  return mod.predict(pipe.transform(x))

pipeline = Pipeline([('encode', ce.OneHotEncoder(use_cat_names=True)),
                     ('power', PowerTransformer_df()),
                     ('transform', StandardScaler_df()),
                     ('feature_extract', PandasFeatureUnion([('interact', PolynomialFeatures_df()), ('bins', KBinsDiscretizer_df(bin_cols))])),
                     ('drop', DropFeatures())])
m = model(**params)

remove_cols = []
# params["class_weight"] = 'balanced'
params["class_weight"] = {}
for i in codes:
  t = 1
  for j in codes:
    if i != j:
      t *= (y_train == j).sum().values[0]
  params["class_weight"][i] = t

while True:
  pipeline.set_params(drop__cols=remove_cols)
  m.fit(pipeline.fit(X_train,y_train).transform(X_train),y_train)
  
  y_pred1 = predict(pipeline,m,X_train)
  y_pred2 = predict(pipeline,m,X_val)
  print("Score:      {: 8.4f}\t{: 8.4f}\t{}".format(scorer(y_train, y_pred1), scorer(y_val, y_pred2), pipeline.named_steps['drop'].max - len(pipeline.named_steps['drop'].cols)))
  p_vals1 = calc_p(pipeline, m, X_val, y_val, predict(pipeline,m,X_val))
  p_vals2 = calc_p(pipeline, m, X_train, y_train, predict(pipeline,m,X_train))
  rem = [p_vals1.index[i] for i in range(len(p_vals1)) if p_vals1.index[i] != "Intercept" and (p_vals1[i] > 0.05 or p_vals1.isna()[i] or p_vals2[i] > 0.05 or p_vals2.isna()[i])]
  if len(rem) == 0:
    break
  remove_cols += rem
  remove_cols = sorted(list(set(remove_cols)))
print()

pipeline.set_params(drop__cols=remove_cols)
model_pipe = Pipeline([('best', SelectKBest(k=i)), ('model', model(**params))])
val_scores = []
for i in range(1,min(pipeline.named_steps['drop'].max - len(pipeline.named_steps['drop'].cols),10)+1):
  model_pipe.set_params(best__k=i)
  model_pipe.fit(pipeline.fit(X_train,y_train).transform(X_train),y_train)
  y_pred1 = predict(pipeline,model_pipe,X_train)
  y_pred2 = predict(pipeline,model_pipe,X_val)
  val_scores.append(scorer(y_val, y_pred2))
  print("Score:      {: 8.4f}\t{: 8.4f}\t{}".format(scorer(y_train, y_pred1), scorer(y_val, y_pred2), "{}/{}".format(i,pipeline.named_steps['drop'].max - len(pipeline.named_steps['drop'].cols))))
print()

ind = val_scores.index(max(val_scores))+1
model_pipe.set_params(best__k=ind)
model_pipe.fit(pipeline.fit(X_train,y_train).transform(X_train),y_train)

y_pred = predict(pipeline,model_pipe,X_test)
print("Final Score:{: 8.4f}\t{:8s}\t{}".format(scorer(y_train, y_pred1), "", "{}/{}".format(val_scores.index(max(val_scores))+1, pipeline.named_steps['drop'].max - len(pipeline.named_steps['drop'].cols))))

ValueError: ignored

In [0]:
p_vals = calc_p(pipeline, m, X_test, y_test, y_pred)
p_vals_train = calc_p(pipeline, m, X_train, y_train, predict(pipeline,model_pipe,X_train))

coef_table = None
colX = pipeline.transform(X_train)

for i in range(len(y_col)):
  if i == 0:
    coef_table = p_vals.to_frame(name="p_{}".format(i))
  else:
    coef_table = coef_table.join(p_vals.to_frame(name="p_{}".format(i)))
  coef_table = coef_table.join(p_vals_train.to_frame(name="p_{} (Train)".format(i)))
  coef_table = coef_table.join(pd.Series(np.append(model_pipe.named_steps["model"].intercept_[i], model_pipe.named_steps["model"].coef_[i]), index=["Intercept"]+[x for ind,x in enumerate(colX) if model_pipe.named_steps["best"].get_support()[ind]]).to_frame(name="Coefficient_{}".format(i)))
  coef_table["abs(Coefficient_{})".format(i)] = coef_table["Coefficient_{}".format(i)].abs()

coef_l = [coef_table.index[ind+1] for ind,x in enumerate(model_pipe.named_steps["best"].get_support()) if x]
coef_table = coef_table.loc[["Intercept"]+coef_l].sort_values(by=["p_0", "abs(Coefficient_0)"], ascending=[True, False])
pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.expand_frame_repr', True)
coef_table

In [0]:
import matplotlib.pyplot as plt

fpr,tpr,_ = roc_curve(y_test, model_pipe.predict_proba(pipeline.transform(X_test))[:,-1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.plot(fpr, tpr)
ax.fill_between(fpr, [0.0]*len(tpr), tpr, alpha=0.5)
ax.set_xlim((0.0,1.0))
ax.set_ylim((0.0,1.0))
plt.show()

print("ROC AUC:", roc_auc_score(y_test, model_pipe.predict_proba(pipeline.transform(X_test))[:,-1]))